Modèle de réseau de neurones sur le dataset du Joueur bas - Henri, Minh, Maxime
- Coup droit
- Déplacement
- Revers
- Service

In [89]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder
from keras import optimizers
from keras import losses
from keras import metrics

In [90]:
# Configuration
path_dataset = 'img/JBas'

# Réseau de neurones classique

## Prises des frames et des vidéos

In [91]:
list_videos = []

In [92]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [93]:
list_videos

In [94]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup dro

In [95]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [96]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [97]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [98]:
# Résolution des images
input_shape_model = list_videos[0].frames.shape[0] * list_videos[0].frames.shape[1] * list_videos[0].frames.shape[2]
input_shape_model

37500

In [99]:
len(list_videos)

214

In [100]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [101]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(171, 171, 43, 43)

In [102]:
paths_test

['img/JBas\\coup droit\\1980-2563441\\images',
 'img/JBas\\coup droit\\1980-2563523\\images',
 'img/JBas\\coup droit\\42698-4288095\\images',
 'img/JBas\\coup droit\\a36r805\\images',
 'img/JBas\\coup droit\\Alcaraz_Sinner270\\images',
 'img/JBas\\coup droit\\AS_13416\\images',
 'img/JBas\\coup droit\\AS_22373\\images',
 'img/JBas\\coup droit\\AS_30502\\images',
 'img/JBas\\coup droit\\AS_37288\\images',
 'img/JBas\\coup droit\\AS_40159\\images',
 'img/JBas\\coup droit\\car0001-0342253\\images',
 'img/JBas\\coup droit\\car5493-5847185\\images',
 'img/JBas\\coup droit\\na0201977\\images',
 'img/JBas\\coup droit\\uopc1041\\images',
 'img/JBas\\coup droit\\uopc1683\\images',
 'img/JBas\\coup droit\\uopc339\\images',
 'img/JBas\\deplacement\\a36r1216\\images',
 'img/JBas\\deplacement\\a36r1713\\images',
 'img/JBas\\deplacement\\a36r172\\images',
 'img/JBas\\deplacement\\a36r257\\images',
 'img/JBas\\deplacement\\a36r339\\images',
 'img/JBas\\deplacement\\AS_13292\\images',
 'img/JBas\\depl

In [103]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [104]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [105]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [106]:
# Normalisation
X_train = X_train / 255.0
X_test = X_test / 255.0

## Modèle de réseau de neurones avec classification multiclass

In [107]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = input_shape_model), # 50*50 * 15 = 37500
        # Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)          
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 32)                1200032   
                                                                 
 dense_16 (Dense)            (None, 64)                2112      
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dense_18 (Dense)            (None, 4)                 516       
                                                                 
Total params: 1,210,980
Trainable params: 1,210,980
Non-trainable params: 0
_________________________________________________________________


In [108]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [109]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

history = model.fit(
    X_train, y_train,
    epochs = 20,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/20
1/5 [=====>........................] - ETA: 1s - loss: 1.3837 - sparse_categorical_accuracy: 0.3750
Epoch 1: val_loss improved from inf to 1.68214, saving model to checkpoints\Weights-001--1.68214.hdf5
5/5 [==============================] - 1s 69ms/step - loss: 1.3761 - sparse_categorical_accuracy: 0.3676 - val_loss: 1.6821 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/20
1/5 [=====>........................] - ETA: 0s - loss: 1.3026 - sparse_categorical_accuracy: 0.4375
Epoch 2: val_loss did not improve from 1.68214
5/5 [==============================] - 0s 23ms/step - loss: 1.2711 - sparse_categorical_accuracy: 0.3676 - val_loss: 1.8609 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/20
1/5 [=====>........................] - ETA: 0s - loss: 1.2605 - sparse_categorical_accuracy: 0.3750
Epoch 3: val_loss did not improve from 1.68214
5/5 [==============================] - 0s 24ms/step - loss: 1.1988 - sparse_categorical_accuracy: 0.3897 - val_loss: 1.9903 - val_s

In [110]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 1.2803 - sparse_categorical_accuracy: 0.3488


[1.2802684307098389, 0.3488371968269348]

In [68]:
pred = model.predict(X_test)
pred

2/2 [==============================] - 0s 5ms/step


array([[0.28639475, 0.51444954, 0.17073978, 0.02841597],
       [0.33815065, 0.3913074 , 0.23877911, 0.03176285],
       [0.3544661 , 0.45454392, 0.16265403, 0.02833588],
       [0.3341438 , 0.45314983, 0.18287466, 0.02983169],
       [0.32503226, 0.4786257 , 0.16444069, 0.03190125],
       [0.37055272, 0.40622208, 0.19302982, 0.03019539],
       [0.42032325, 0.32195306, 0.22464113, 0.03308259],
       [0.47009557, 0.30990306, 0.18372357, 0.03627783],
       [0.2990541 , 0.5035109 , 0.16715892, 0.03027602],
       [0.37917268, 0.42190105, 0.16863847, 0.03028771],
       [0.4184212 , 0.3466131 , 0.20369585, 0.03126984],
       [0.4739001 , 0.31713077, 0.17837945, 0.0305897 ],
       [0.34033954, 0.45189917, 0.17891619, 0.02884514],
       [0.4034711 , 0.36759067, 0.1962039 , 0.03273435],
       [0.36400867, 0.43748397, 0.16947964, 0.02902766],
       [0.4165887 , 0.3620459 , 0.1902865 , 0.03107897],
       [0.34564933, 0.42356148, 0.20090973, 0.02987945],
       [0.33287156, 0.47263396,

In [69]:
y_pred = np.argmax(pred, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0
 0 1 0 0 1 0]
True labels:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 3 3 3 3]


**Accuracy: $17 / 43$ ($39.53 \%$)**

# Réseaux de neurones convolutifs

## Prises des frames et des vidéos

In [70]:
list_videos_cnn = []
y_cnn = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    vid = Video.read_video_from_frames(frames)
                    list_videos_cnn.append(vid)
                    y_cnn.append(output_res)

In [71]:
list_videos_cnn[0].frames.shape[1:]

(50, 50)

In [72]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [73]:
y_cnn = encoder.fit_transform(y_cnn)
y_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [74]:
output_y = np.unique(y_cnn)
output_y

array([0, 1, 2, 3], dtype=int64)

In [75]:
len(list_videos_cnn)

214

In [76]:
train_size = int(len(list_videos_cnn) * 0.8)
test_size = len(list_videos_cnn) - train_size

In [77]:
## Split video train, test
random.seed(42)
vids_train_cnn = random.sample(list_videos_cnn, k = train_size)
vids_test_cnn = [vid for vid in list_videos_cnn if vid not in vids_train_cnn]
y_train_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_train_cnn]
y_test_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_test_cnn]
len(vids_train_cnn), len(y_train_cnn), len(vids_test_cnn), len(y_test_cnn)

(171, 171, 43, 43)

In [78]:
y_train_cnn = np.array(y_train_cnn, dtype=int)
y_test_cnn = np.array(y_test_cnn, dtype=int)
y_train_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [79]:
# Split input X en train, test set
X_train_cnn = np.zeros((train_size, 50, 15*50, 1), dtype=int)
for i, vid in enumerate(vids_train_cnn):
    for j, frame in enumerate(vid.frames):
        X_train_cnn[i, :, 50*j:50*(j+1), :] = frame.reshape(50, 50, 1)
X_train_cnn.shape

(171, 50, 750, 1)

In [80]:
X_test_cnn = np.zeros((test_size, 50, 15*50, 1), dtype=int)
for i, vid in enumerate(vids_test_cnn):
    for j, frame in enumerate(vid.frames):
        X_test_cnn[i, :, 50*j:50*(j+1), :] = frame.reshape(50, 50, 1)
X_test_cnn.shape

(43, 50, 750, 1)

In [81]:
input_shape_cnn = X_train_cnn.shape[1:]
input_shape_cnn

(50, 750, 1)

## Modèle de réseau de neurones convolutifs

In [82]:
model_cnn = Sequential(
    [
        tf.keras.Input(shape = input_shape_cnn), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 748, 16)       160       
                                                                 
 batch_normalization (BatchN  (None, 48, 748, 16)      64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 374, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 374, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 372, 32)       4640      
                                                                 
 batch_normalization_1 (Batc  (None, 22, 372, 32)     

In [83]:
checkpoint_name = 'checkpoints/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [84]:
model_cnn.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

history_cnn = model_cnn.fit(
    X_train_cnn, y_train_cnn,
    epochs = 20,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/20
5/5 [==============================] - ETA: 0s - loss: 2.4379 - sparse_categorical_accuracy: 0.2500
Epoch 1: val_loss improved from inf to 67.27535, saving model to checkpoints\Weights-001--67.27535.hdf5
5/5 [==============================] - 4s 662ms/step - loss: 2.4379 - sparse_categorical_accuracy: 0.2500 - val_loss: 67.2754 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/20
5/5 [==============================] - ETA: 0s - loss: 1.3056 - sparse_categorical_accuracy: 0.4632
Epoch 2: val_loss improved from 67.27535 to 62.25179, saving model to checkpoints\Weights-002--62.25179.hdf5
5/5 [==============================] - 3s 609ms/step - loss: 1.3056 - sparse_categorical_accuracy: 0.4632 - val_loss: 62.2518 - val_sparse_categorical_accuracy: 0.0857
Epoch 3/20
5/5 [==============================] - ETA: 0s - loss: 0.8877 - sparse_categorical_accuracy: 0.5956
Epoch 3: val_loss improved from 62.25179 to 56.84739, saving model to checkpoints\Weights-003--56.84739.hdf5
5/5 [====

In [86]:
model_cnn.evaluate(X_test_cnn, y_test_cnn)

2/2 [==============================] - 0s 81ms/step - loss: 4.7984 - sparse_categorical_accuracy: 0.3256


[4.798425197601318, 0.3255814015865326]

In [87]:
pred_cnn = model_cnn.predict(X_test_cnn)
pred_cnn

2/2 [==============================] - 0s 101ms/step


array([[4.98157134e-03, 9.93046939e-01, 1.97145552e-03, 1.31618317e-19],
       [3.64651293e-01, 3.11381947e-02, 6.04210496e-01, 2.94653496e-20],
       [1.55160278e-01, 3.87292802e-01, 4.57546949e-01, 1.45196161e-18],
       [3.48140448e-02, 9.45998728e-01, 1.91872548e-02, 2.95871235e-19],
       [9.79459845e-03, 9.87996161e-01, 2.20929482e-03, 2.52888039e-16],
       [4.95618820e-01, 1.98266715e-01, 3.06114554e-01, 1.51087103e-17],
       [7.00257346e-03, 5.98267373e-03, 9.87014651e-01, 2.94491249e-18],
       [9.99017715e-01, 3.98492382e-04, 5.83746063e-04, 7.97543989e-17],
       [2.44852472e-02, 7.20003009e-01, 2.55511761e-01, 9.20889423e-19],
       [7.81272113e-01, 1.99306577e-01, 1.94212776e-02, 2.75326343e-19],
       [7.62522042e-01, 7.54889622e-02, 1.61989033e-01, 6.51040411e-19],
       [9.39615726e-01, 5.77290207e-02, 2.65534222e-03, 3.50130664e-18],
       [2.74589837e-01, 6.88552082e-01, 3.68581191e-02, 1.43692468e-20],
       [4.22382474e-01, 3.81495446e-01, 1.96122110e

In [88]:
y_pred_cnn = np.argmax(pred_cnn, axis=1)
print("Predicted labels: ", y_pred_cnn)
print("True labels:      ", y_test_cnn)

Predicted labels:  [1 2 2 1 1 0 2 0 1 0 0 0 1 0 1 0 0 1 1 1 0 0 2 0 1 1 0 1 0 0 2 2 2 0 0 0 1
 0 1 0 1 0 2]
True labels:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 3 3 3 3]


**Accuracy: $20/43$ ($46.51 \%$)**